In [1]:
using Pkg
#Pkg.add("RigidBodyDynamics")
#Pkg.add("MeshCat")
#Pkg.add("MeshCatMechanisms")
#Pkg.add("StaticArrays")
#Pkg.add("Rotations")
#Pkg.add("ForwardDiff")
#Pkg.add("FiniteDiff")
#Pkg.add("Quaternions")
#Pkg.add("OSQP")
#Pkg.add("Plots")
#Pkg.add("Colors")
#Pkg.add("MathOptInterface")
#Pkg.add("Ipopt")
#Pkg.add("GeometryBasics")
#Pkg.add("CoordinateTransformations")
Pkg.add("Optim")
include(joinpath(@__DIR__, "humanoid.jl"))
include(joinpath(@__DIR__, "ball_qp.jl"))
using LinearAlgebra
import ForwardDiff as FD
using GeometryBasics
using Optim

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


WebIO._IJuliaInit()

## Finding Robot Balancing Pose

In [2]:
model = G1Humanoid()
mech = model.mech
const x_guess = initial_state(model) # our guess state for balancing
mvis = initialize_visualizer(model) # visualizer 
set_configuration!(mvis, x_guess[1:state_dim(model)÷2])
foot_pos = get_right_foot_tip_location(mech, x_guess)
println("Foot tip position: ", foot_pos)

# Add red sphere at foot location
foot_point = mvis.visualizer["foot_point"]
setobject!(
    foot_point, 
    HyperSphere(Point3f(foot_pos), Float32(0.01))  # Both Float32
)
setprop!(foot_point, "color", RGB(1, 0, 0))  # Red color
render(mvis)

[ Info: Listening on: 127.0.0.1:8729, thread id: 1


Foot tip position: 

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8729


[-0.4539735038572158, -0.19194453008619367, 0.2135702686910089]


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("foot_point" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x66, 0x6f, 0x6f, 0x74, 0x5f, 0x70, 0x6f, 0x69, 0x6e, 0x74], nothing, Dict{String, Vector{UInt8}}("color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x66, 0x6f, 0x6f, 0x74, 0x5f, 0x70, 0x6f, 0x69, 0x6e, 0x74]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("dummy1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("dummy2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6d, 0x79, 0x31, 0x2f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("left_ankle_roll_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x72, 0x6f, 0x6c, 0x6c, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_left_ankle_roll_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x6c, 0x6c, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_8" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x38], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x38], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "left_ankle_pitch_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x70, 0x69, 0x74, 0x63, 0x68, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_left_ankle_pitch_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x74, 0x63, 0x68, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_7" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x37], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x37], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "left_knee_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x6b, 0x6e, 0x65, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_left_knee_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x6e, 0x65, 0x65, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_6" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65,

In [3]:
function linesearch(z::Vector, Δz::Vector, merit_fx::Function;
                    max_ls_iters = 10)::Float64 # optional argument with a default

    α = 1
    for i = 1:max_ls_iters
        
        if (merit_fx(z + α * Δz) < merit_fx(z))
            return α
        end
        α = α / 2
        
    end
    error("linesearch failed")
end

function newtons_method(z0::Vector, res_fx::Function, res_jac_fx::Function, merit_fx::Function;
                        tol = 1e-10, max_iters = 50, verbose = false)::Vector{Vector{Float64}}
    
    # Computes Newton's method given the following inputs:
    # - z0, initial guess 
    # - res_fx, residual function 
    # - res_jac_fx, Jacobian of residual function wrt z 
    # - merit_fx, merit function for use in linesearch 
    
    # optional arguments 
    # - tol, tolerance for convergence. Return when norm(residual)<tol 
    # - max iter, max # of iterations 
    # - verbose, bool telling the function to output information at each iteration
    
    # return a vector of vectors containing the iterates 
    # the last vector in this vector of vectors should be the approx. solution 
    
    # return the history of guesses as a vector
    Z = [zeros(length(z0)) for i = 1:max_iters]
    Z[1] = z0 
    
    for i = 1:(max_iters - 1)        
        r = res_fx(Z[i]) 
        
        norm_r = norm(r) 
        if verbose 
            print("iter: $i    |r|: $norm_r   ")
        end
        
        # if converged, return Z[1:i]
        if norm_r < tol
            return Z[1:i]
        end
        
        Δz = -1 *  res_jac_fx(Z[i]) \ res_fx(Z[i])
        
        α = linesearch(Z[i], Δz, merit_fx)

        Z[i+1] = Z[i] .+ α*Δz

        if verbose
            print("α: $α \n")
        end
        
    end
    error("Newton's method did not converge")
end
function merit(z)
    # merit function
    r = kkt(z)
    return norm(r[1:94]) + 1e4*norm(r[94:end])
end

merit (generic function with 1 method)

In [4]:
# initial guess 
const x_guess = initial_state(model)

# indexing stuff 
const idx_x = 1:64
const idx_u = 65:93
const idx_c = 94:(157 + 3)

function cost(y::Vector)
    x = y[idx_x]
    u = y[idx_u]
    
    return 1/2 * (x - x_guess)' * (x - x_guess) + 1/2 * 1e-3*u'*u
end
function constraint(y::Vector)::Vector
    x = y[idx_x]
    u = y[idx_u]

    dynamics_constraint = dynamics(model, x, u)
    floot_flat_constraint = x[1:3]
    return [dynamics_constraint; floot_flat_constraint]
end
function kkt(z::Vector)::Vector
    x = z[idx_x]
    u = z[idx_u]
    λ = z[idx_c]
    
    y = [x;u]
    s = FD.gradient(cost, y) + FD.jacobian(constraint, y)' * λ
    
    pf = constraint(y)
    return [s; pf]
end

function kkt_jac(z::Vector)::Matrix
    x = z[idx_x]
    u = z[idx_u]
    λ = z[idx_c]
    
    y = [x;u]
    
    H = FD.hessian(cost, y)
    C = FD.jacobian(constraint, y)
    J = [
        H C';
        C zeros(size(λ, 1), size(λ, 1))
    ]

    β = 1e-3
    reg = [I(size(y, 1)) zeros(size(C')); zeros(size(C)) I(size(λ, 1))]

    return J + β * reg
    
end
   
    
z0 = [x_guess; zeros(29); zeros(67)]
Z = newtons_method(z0, kkt, kkt_jac, merit; tol = 1e-6, verbose = false, max_iters = 50)
# visualizer 
set_configuration!(mvis, Z[end][1:state_dim(model)÷2])
u_eq = Z[end][idx_u]
x_eq = Z[end][idx_x]
x_dot = dynamics(model, x_eq, u_eq)
@assert norm(x_dot, Inf) < 1e-6
foot_equilib_pos = get_right_foot_tip_location(mech, x_eq)
println(foot_equilib_pos)
println("Equilibrium found! $(norm(x_dot, Inf))")
render(mvis)

[-0.36974168511631605, -0.18129216838310608, 0.28428794901543925]
Equilibrium found! 6.824357431475766e-12


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("foot_point" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x66, 0x6f, 0x6f, 0x74, 0x5f, 0x70, 0x6f, 0x69, 0x6e, 0x74], nothing, Dict{String, Vector{UInt8}}("color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x66, 0x6f, 0x6f, 0x74, 0x5f, 0x70, 0x6f, 0x69, 0x6e, 0x74]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("dummy1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("dummy2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6d, 0x79, 0x31, 0x2f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("left_ankle_roll_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x72, 0x6f, 0x6c, 0x6c, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_left_ankle_roll_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x6c, 0x6c, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_8" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x38], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x38], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "left_ankle_pitch_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x70, 0x69, 0x74, 0x63, 0x68, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_left_ankle_pitch_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x74, 0x63, 0x68, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_7" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x37], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x37], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "left_knee_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x6b, 0x6e, 0x65, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_left_knee_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x6e, 0x65, 0x65, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_6" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65,

In [5]:
x_max = norm(x_eq, Inf)
u_max = norm(u_eq, Inf)
println(x_max)
println(u_max)
save("equilibrium.jld2", "x", x_eq, "u", u_eq)


1.4157584329238388
13.891208669154103


## Solve For Initial Guess

In [5]:
function generate_kick_arc_trajectory(
    start_pos::Vector{Float64},
    end_pos::Vector{Float64},
    arc_dip::Float64,
    arc_rise::Float64,
    max_step_norm::Float64;
    max_points::Int = 500,
    scale_dip_if_needed::Bool = true
)
    num_points = 2
    orig_dip = arc_dip  # Save original for scaling logic

    function create_kick_arc(n, dip)
        arc_traj = Vector{SVector{3, Float64}}()
        for i in range(0, 1, length=n)
            pos = (1 - i) * start_pos .+ i * end_pos

            if i < 0.5
                z_mod = -4 * dip * i * (1 - i)
            else
                z_mod = 2 * arc_rise * (i - 0.5)^2
            end

            pos[3] += z_mod
            push!(arc_traj, SVector{3}(pos))
        end
        return arc_traj
    end

    while num_points <= max_points
        arc = create_kick_arc(num_points, arc_dip)
        max_dist = maximum(norm(arc[i+1] - arc[i]) for i in 1:length(arc)-1)

        if max_dist < max_step_norm
            return arc
        end

        # Optional: adaptive dip scaling
        if scale_dip_if_needed && num_points == max_points
            arc_dip *= 0.95  # reduce dip a bit
            num_points = 2   # restart with fewer points
            #@info "Reducing dip to $(round(arc_dip, digits=4)) to meet step constraint."
        else
            num_points += 1
        end
    end

    error("Could not satisfy step constraint after $max_points points. Final dip = $(round(arc_dip, digits=4))")
end




# Example inputs
equilib_foot_pos = [-0.3697416851162835, -0.1812921683831133, 0.28428794901541954]
goal_foot_pos = [0.1, -0.13, 0.07]

kick_arc = generate_kick_arc_trajectory(
    equilib_foot_pos, 
    goal_foot_pos, 
    0.05,   
    0.01,
    0.02
)

if kick_arc[size(kick_arc, 1)] != goal_foot_pos
    push!(kick_arc, goal_foot_pos)
end

if kick_arc[1] == equilib_foot_pos
    popfirst!(kick_arc)
end
# Print the resulting arc
for pos in kick_arc
    println(pos)
end

[-0.35669330497416457, -0.1798673859280268, 0.2760771399790852]
[-0.34364492483204556, -0.17844260347294033, 0.26799538042892956]
[-0.33059654468992655, -0.17701782101785385, 0.2600426703649527]
[-0.31754816454780754, -0.17559303856276737, 0.25221900978715456]
[-0.3044997844056886, -0.17416825610768089, 0.24452439869553533]
[-0.29145140426356964, -0.17274347365259443, 0.23695883709009471]
[-0.2784030241214506, -0.17131869119750795, 0.22952232497083289]
[-0.2653546439793317, -0.16989390874242144, 0.2222148623377498]
[-0.2523062638372126, -0.16846912628733499, 0.21503644919084552]
[-0.23925788369509365, -0.1670443438322485, 0.20798708553011996]
[-0.22620950355297464, -0.165619561377162, 0.2010667713555732]
[-0.21316112341085572, -0.16419477892207554, 0.19427550666720517]
[-0.20011274326873668, -0.16276999646698903, 0.18761329146501587]
[-0.18706436312661773, -0.16134521401190258, 0.1810801257490054]
[-0.17401598298449872, -0.15992043155681607, 0.17467600951917361]
[-0.16096760284237974, 

In [6]:
x_curr = x_eq
states = Vector{Vector{Float64}}(undef, length(kick_arc)+1)
states[1] = x_eq
k = 2
for desired_foot_pos in kick_arc
    # indexing stuff 
    idx_x = 1:64
    idx_u = 65:93
    idx_c = 94:(157 + 3 + 3)
    function cost(y::Vector)
        x = y[idx_x]
        u = y[idx_u]
        
        return 1/2 * (x - x_curr)' * (x - x_curr) + 1/2 * 1e-3*u'*u
    end
    function constraint(y::Vector)::Vector
        x = y[idx_x]
        u = y[idx_u]

        dynamics_constraint = dynamics(model, x, u)
        floot_flat_constraint = x[1:3]
        kick_constraint = get_right_foot_tip_location(model.mech, x) - desired_foot_pos
        return [dynamics_constraint; floot_flat_constraint; kick_constraint]
    end
    function kkt(z::Vector)::Vector
        x = z[idx_x]
        u = z[idx_u]
        λ = z[idx_c]
        
        y = [x;u]
        s = FD.gradient(cost, y) + FD.jacobian(constraint, y)' * λ
        
        pf = constraint(y)
        return [s; pf]
    end

    function kkt_jac(z::Vector)::Matrix
        x = z[idx_x]
        u = z[idx_u]
        λ = z[idx_c]
        
        y = [x;u]
        
        H = FD.hessian(cost, y)
        C = FD.jacobian(constraint, y)
        J = [
            H C';
            C zeros(size(λ, 1), size(λ, 1))
        ]

        β = 1e-3
        reg = [I(size(y, 1)) zeros(size(C')); zeros(size(C)) I(size(λ, 1))]

        return J + β * reg
        
    end
    println("Solving timestep $(k), with foot position: $(desired_foot_pos)")
    z0 = [x_curr; zeros(29); zeros(70)]
    Z = newtons_method(z0, kkt, kkt_jac, merit; tol = 1e-3, verbose = false, max_iters = 1000)
    u = Z[end][idx_u]
    x_curr = Z[end][idx_x]
    states[k] = x_curr
    k += 1
end


Solving timestep 2, with foot position: [-0.35669330497416457, -0.1798673859280268, 0.2760771399790852]


Solving timestep 3, with foot position: [-0.34364492483204556, -0.17844260347294033, 0.26799538042892956]
Solving timestep 4, with foot position: [-0.33059654468992655, -0.17701782101785385, 0.2600426703649527]
Solving timestep 5, with foot position: [-0.31754816454780754, -0.17559303856276737, 0.25221900978715456]
Solving timestep 6, with foot position: [-0.3044997844056886, -0.17416825610768089, 0.24452439869553533]
Solving timestep 7, with foot position: [-0.29145140426356964, -0.17274347365259443, 0.23695883709009471]
Solving timestep 8, with foot position: [-0.2784030241214506, -0.17131869119750795, 0.22952232497083289]
Solving timestep 9, with foot position: [-0.2653546439793317, -0.16989390874242144, 0.2222148623377498]
Solving timestep 10, with foot position: [-0.2523062638372126, -0.16846912628733499, 0.21503644919084552]
Solving timestep 11, with foot position: [-0.23925788369509365, -0.1670443438322485, 0.20798708553011996]
Solving timestep 12, with foot position: [-0.226209

In [ ]:
mvis = initialize_visualizer(model) # visualizer
open(mvis)
dt = 0.05
for k = 1:length(states)
    q = states[k]
    v = zeros(length(q))              # zero velocities
    full_state = [q; v]               # concatenate position and velocity
    set_configuration!(mvis, full_state[1:state_dim(model)÷2])
    sleep(dt)
    render(mvis)
end

[ Info: Listening on: 127.0.0.1:8737, thread id: 1
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8737


In [ ]:
for k = 1:length(kick_arc)-1
    states[k][33:64] = (states[k+1][1:32] - states[k][1:32]) / dt
end

nv = 32

window_size = 3
states[end][33:64] = states[end-1][33:64]
for i in 1:length(kick_arc)
    count = 0
    for j in max(1, i - window_size):min(length(kick_arc), i + window_size)
        states[i][33:64] .+= states[j][33:64]
        count += 1
    end
    states[i][33:64] ./= count
end
save("guess.jld2", "x", states)

38

In [46]:
using RigidBodyDynamics

state = MechanismState(model.mech)
v̇ = similar(velocity(state))  # SegmentedVector{jointid} for joint accelerations
result = DynamicsResult(model.mech)

u_list = Vector{Vector{Float64}}(undef, length(states)-1)

nq = 32
nv = 32
for k in 1:length(kick_arc) - 1
    x1 = states[k]
    x2 = states[k+1]

    # Extract states
    q = x1[1:nq]
    v = x1[nq+1:nq+nv]
    v_next = x2[nq+1:nq+nv]
    a = (v_next - v) / dt
    #println(maximum(a))
    set_configuration!(state, q)
    set_velocity!(state, v)
    copyto!(v̇, a)
    u = inverse_dynamics(state, v̇) # requires segement vector input
    u_list[k] = copy(u)
end

In [47]:
function hermite_simpson(model, x1, x2, u, dt::Real)
    x1dot = dynamics(model, x1, u[4:end])
    x2dot = dynamics(model, x2, u[4:end])
    x_k12 = 1/2 * (x1 + x2) + dt/8 * (x1dot - x2dot)
    #@show norm(x1dot), norm(x2dot), norm(x_k12)
    return x1 + dt/6 * (x1dot + 4 * dynamics(model, x_k12, u[4:end]) + x2dot) - x2
end

x_discrete = Vector{Vector{Float64}}(undef, length(states)-1)
x_discrete[1] = x_eq
println(x_eq)
for k = 1:length(kick_arc) - 1
    xp1 = hermite_simpson(model, states[k], states[k+1], u_list[k], dt)
    x_discrete[k+1] = xp1
    println(xp1)
end

mvis = initialize_visualizer(model) # visualizer
open(mvis)
set_configuration!(mvis, x_discrete[2][1:state_dim(model)÷2])
render(mvis)

# open(mvis)
# for k = 1:length(kick_arc)
#     x = x_discrete[k]
#     set_configuration!(mvis, x[1:state_dim(model)÷2])
#     sleep(dt)
#     render(mvis)
# end

[1.9206790386412052e-12, 2.808912675253687e-12, 1.677233253661875e-15, 0.055885794070459426, -0.08481177117166601, 0.24759979934721957, -0.09589945974351341, 0.5221884661892939, -0.45625849809228763, -0.3203508745543521, 0.005079869213525887, 0.19599109240866744, 0.10910769293267471, -0.10318426248232228, -0.17245876410021446, 1.4157584329238388, 0.20913638218966316, 0.2077991258851042, 0.17509396581556846, 0.07785586814903218, 0.07743992433453438, 0.00134817892116859, 0.08356762924822243, 0.08209750526819433, -0.5253144489708447, -0.525542899213209, 0.1744995706828743, 0.1745139907221226, -0.0007927339886340707, -0.0008594514354240303, -0.00023311554445147912, -0.0003484705129092962, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-0.0018498307304751751, -0.015616488318149229, 0.012490210079777245, 0.0014300459945057534, -0.042038343725335874, 0.0592530598373448, -0.0066656

[ Info: Listening on: 127.0.0.1:8738, thread id: 1
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8738


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("dummy1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("dummy2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6d, 0x79, 0x31, 0x2f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("left_ankle_roll_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x72, 0x6f, 0x6c, 0x6c, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_left_ankle_roll_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x6c, 0x6c, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_8" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x38], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x38], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "left_ankle_pitch_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x70, 0x69, 0x74, 0x63, 0x68, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_left_ankle_pitch_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x74, 0x63, 0x68, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_7" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x37], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x37], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "left_knee_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x6b, 0x6e, 0x65, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_left_knee_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x6e, 0x65, 0x65, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_6" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x36], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x36], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "left_hip_yaw_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x70, 0x5f, 0x79, 0x61, 

## Ball Dynamics, Calculating Kick Point

In [ ]:
q_i = [0.0, 0.0] # Note, this is in the X-Z Plane only, y value will be assigned manually after the fact
q_des = [20.0, 3.0]
J = optimize_impulse(q_des, q_i) # Can also specify mass
dynamics_rollout(J, q_des, q_i)
p_c = get_contact_point(J) # Can also specify radius of ball
visualize_contact(J, p_c)

## Calculating Kick Trajectory

In [10]:
using Pkg

#Pkg.add("NamedTrajectories")
#Pkg.add("CairoMakie")
#Pkg.add("BenchmarkTools")
Pkg.add(url="https://github.com/kevin-tracy/lazy_nlp_qd.jl.git")
#Pkg.add("ProgressMeter")
using NamedTrajectories
using CairoMakie
using BenchmarkTools
import ForwardDiff as FD
using LinearAlgebra
using Plots
using lazy_nlp_qd
#import MeshCat as mc
using StaticArrays
using ProgressMeter
using FileIO
include(joinpath(@__DIR__, "../src/utils/utils.jl"))
include(joinpath(@__DIR__, "../src/utils/nlp_utils.jl"))

    Updating git-repo `https://github.com/kevin-tracy/lazy_nlp_qd.jl.git`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


constraint_violation (generic function with 1 method)

In [165]:
function hermite_simpson(params::NamedTuple, x1::Vector, x2::Vector, u)::Vector
    model = params.model
    dt = params.dt
    x1dot = dynamics(model, x1, u)
    x2dot = dynamics(model, x2, u)
    x_k12 = 1/2 * (x1 + x2) + dt/8 * (x1dot - x2dot)
    return x1 + dt/6 * (x1dot + 4 * dynamics(model, x_k12, u) + x2dot) - x2
end

hermite_simpson (generic function with 2 methods)

In [28]:
function quadratic_cost(traj_indices::NamedTuple, k::Int64, Q1::Matrix{Float64}, Q2::Matrix{Float64}, R::Matrix{Float64}, x_g, foot_ref, mech, kick_time)
    xi, ui = traj_indices.x[k], traj_indices.u[k]
    foot_body = findbody(model.mech, "right_ankle_roll_link")
    world_body = findbody(model.mech, "world")
    kinematic_path = path(model.mech, foot_body, world_body)
    desired_foot_pos = foot_ref[k]
    s_p_ic = traj_indices.s_p_ic[k]
    s_n_ic = traj_indices.s_n_ic[k]
    s_p_foot = traj_indices.s_p_foot[k]
    s_n_foot = traj_indices.s_n_foot[k]
    s_p_dynamics = traj_indices.s_p_dynamics[k]
    s_n_dynamics = traj_indices.s_n_dynamics[k]
    # s_p_g = traj_indices.s_p_g[k]
    # s_n_g = traj_indices.s_p_g[k]
    p = 1000
    # TODO: Tune this cost function
    function cost_func(params::NamedTuple, z::Vector)
        x = z[xi]
        foot_tip_pos = get_right_foot_tip_location(mech, x)
        J = 0
        J += p * (sum(abs.(z[s_p_ic])) + sum(abs.(z[s_n_ic])))  # Penalize slack for initial condition
        J += p * (sum(abs.(z[s_p_foot])) + sum(abs.(z[s_n_foot]))) 
        # J += p * (sum(abs.(z[s_p_g])) + sum(abs.(z[s_n_g]))) 
        J += p * (sum(abs.(z[s_p_dynamics])) + sum(abs.(z[s_n_dynamics]))) 
        J += 0.5*(foot_tip_pos - desired_foot_pos)'*Q1*(foot_tip_pos - desired_foot_pos) + 0.5 * z[ui]'*R*z[ui]
        return J
    end

    function cost_grad!(params::NamedTuple, z::Vector{Float64}, grad::Vector{Float64})
        grad .+= FD.gradient(z_ -> cost_func(params, z_), z)
    end
    return (cost_func=cost_func, cost_grad=cost_grad!)
end

function final_cost(traj_indices::NamedTuple, N::Int64, Qf::Matrix{Float64}, foot_ref, mech, x_g)
    xi = traj_indices.x[N]
    # TODO: Tune this cost function
    # s_p = traj_indices.s_p_g[N]
    # s_n = traj_indices.s_p_g[N]
    p = 1000
    s_p_foot = traj_indices.s_p_foot[N]
    s_n_foot = traj_indices.s_n_foot[N]
    desired_foot_pos = foot_ref[N]

    function cost_func(params::NamedTuple, z::Vector)
        foot_tip_pos = get_right_foot_tip_location(mech, z[xi])
        J = p * (sum(abs.(z[s_p_foot])) + sum(abs.(z[s_n_foot]))) 
        return J + 0.5*((foot_tip_pos - desired_foot_pos)'*Qf*(foot_tip_pos - desired_foot_pos))
    end

    function cost_grad!(params::NamedTuple, z::Vector{Float64}, grad::Vector{Float64})
        grad .+= FD.gradient(z_ -> cost_func(params, z_), z)
    end
    return (cost_func=cost_func, cost_grad=cost_grad!)
end

# Dynamics Constraints
function dyn_constraint(traj_indices::NamedTuple, k::Int64)
    # Get indices for knot data in trajectory
    xi, ui, next_xi = traj_indices.x[k], traj_indices.u[k], traj_indices.x[k+1]
    s_pi, s_ni = traj_indices.s_p_dynamics[k], traj_indices.s_n_dynamics[k]
    nx = 64
    function residual(params::NamedTuple, z::Vector, con::AbstractVector)
        con .= hermite_simpson(params, z[xi], z[next_xi], z[ui]) - z[s_pi] + z[s_ni]
    end
    function jacobian!(params::NamedTuple, z::Vector, conjac::AbstractMatrix)
        x_k, u_k, next_x_k = z[xi], z[ui], z[next_xi]
        conjac[:, xi] = FD.jacobian(x_ -> hermite_simpson(params, x_, next_x_k, u_k), x_k);
        conjac[:, ui] = FD.jacobian(u_ -> hermite_simpson(params, x_k, next_x_k, u_), u_k);
        conjac[:, next_xi] = FD.jacobian(x_ -> hermite_simpson(params, x_k, x_, u_k), next_x_k);
        conjac[:, s_pi] = -1 .* I(length(z[s_pi]))
        conjac[:, s_ni] = I(length(z[s_ni]))
        return nothing
    end

    # Indication of which blocks are populated (i.e. sparsity structure) for constraint
    function sparsity!(conjac::AbstractMatrix)
        conjac[:, xi] .= 1;
        conjac[:, ui] .= 1;
        conjac[:, next_xi] .= 1;
        conjac[:, s_pi] .= 1
        conjac[:, s_ni] .= 1
        return nothing
    end
    bounds = (zeros(nx), zeros(nx))
    return (length=nx, residual=residual, jacobian=jacobian!, sparsity=sparsity!, bounds=bounds)
end

# State Intial Condition Constraint
function ic_constraint(traj_indices::NamedTuple, x_ic::Vector{Float64})
    x0 = traj_indices.x[1]
    s_p, s_n = traj_indices.s_p_ic[1], traj_indices.s_n_ic[1]
    nx = 64
    function residual(params::NamedTuple, z::Vector, con::AbstractVector) 
        con .= z[x0] - x_ic - z[s_p] + z[s_n]
    end
    function jacobian!(params::NamedTuple, z::Vector, conjac::AbstractMatrix)
        conjac[:, x0] = I(length(x_ic))
        conjac[:, s_p] = -1 .* I(length(z[s_p]))
        conjac[:, s_n] = I(length(z[s_n]))
        return nothing
    end
    function sparsity!(conjac::AbstractMatrix)
        conjac[:, x0] .= 1
        conjac[:, s_p] .= 1
        conjac[:, s_n] .= 1
        return nothing
    end
    bounds = (zeros(nx), zeros(nx))
    return (length=nx, residual=residual, jacobian=jacobian!, sparsity=sparsity!, bounds=bounds)
end

# State Goal Condition Constraint
# function goal_constraint(traj_indices::NamedTuple, x_g::Vector{Float64}, N)
#     x_N = traj_indices.x[N]
#     s_p, s_n = traj_indices.s_p_g[N], traj_indices.s_n_g[N]
#     nx = 64
#     function residual(params::NamedTuple, z::Vector, con::AbstractVector)
#         con .= z[x_N] - x_g - z[s_p] + z[s_n]
#     end
#     function jacobian!(params::NamedTuple, z::Vector, conjac::AbstractMatrix)
#         conjac[:, x_N] = I(length(x_N))
#         conjac[:, s_p] = -1 .* I(length(z[s_p]))
#         conjac[:, s_n] = I(length(z[s_n]))
#         return nothing
#     end
#     function sparsity!(conjac::AbstractMatrix)
#         conjac[:, x_N] .= 1
#         conjac[:, s_p] .= 1
#         conjac[:, s_n] .= 1
#     end
#     bounds = (zeros(nx), zeros(nx))
#     return (length=nx, residual=residual, jacobian=jacobian!, sparsity=sparsity!, bounds=bounds)
# end

function foot_position_constraint(traj_indices::NamedTuple, mech::Mechanism, ball_pos::AbstractVector, k::Int) # use this as goal constraint as well
    xi = traj_indices.x[k]
    foot_body = findbody(model.mech, "right_ankle_roll_link")
    world_body = findbody(model.mech, "world")
    kinematic_path = path(model.mech, foot_body, world_body)
    s_pi, s_ni = traj_indices.s_p_foot[k], traj_indices.s_n_foot[k]
    residual(params::NamedTuple, z::Vector, con::AbstractVector) = begin
        foot_tip_pos = get_right_foot_tip_location(mech, z[xi])
        con .= foot_tip_pos - ball_pos - z[s_pi] + z[s_ni]
    end

   jacobian!(params::NamedTuple, z::Vector, conjac::AbstractMatrix) = begin
        state = MechanismState(mech)
        copyto!(state, z[xi])
        
        foot_jacobian = geometric_jacobian(state, kinematic_path)
        
        # Extract translation components (3 × nq matrix)
        J_trans = Matrix(foot_jacobian)[4:6, :]
        nq = size(J_trans, 2)
        if length(xi) < nq
            error("xi must span at least $nq columns (joint positions) but has length $(length(xi))")
        end
        conjac[:, xi[1:nq]] .= J_trans
        conjac[:, s_pi] .= -1 .* I(length(z[s_pi]))
        conjac[:, s_ni] .= I(length(z[s_ni]))
        return nothing
    end
    
    
    function sparsity!(conjac::AbstractMatrix)
        conjac[:, xi] .= 1
        conjac[:, s_pi] .= 1
        conjac[:, s_ni] .= 1
        return nothing
    end
    bounds = (zeros(3), zeros(3))
    
    return (length=3, residual=residual, jacobian=jacobian!, sparsity=sparsity!, bounds=bounds)
end

foot_position_constraint (generic function with 1 method)

In [26]:
equilib_loaded = load("equilibrium.jld2")
x_eq = equilib_loaded["x"]
u_eq = equilib_loaded["u"]

lower_limits = [
    -2.5307, -0.5236, -2.7576, -0.087267, -0.87267, -0.2618,
    -2.5307, -2.9671, -2.7576, -0.087267, -0.87267, -0.2618,
    -2.618, -0.52, -0.52,
    -3.0892, -1.5882, -2.618, -1.0472, -1.9722, -1.6144, -1.6144,
    -3.0892, -2.2515, -2.618, -1.0472, -1.9722, -1.6144, -1.6144
]

upper_limits = [
     2.8798,  2.9671,  2.7576,  2.8798,  0.5236,  0.2618,
     2.8798,  0.5236,  2.7576,  2.8798,  0.5236,  0.2618,
     2.618,  0.52,  0.52,
     2.6704,  2.2515,  2.618,  2.0944,  1.9722,  1.6144, 1.6144,
     2.6704,  1.5882,  2.618,  2.0944,  1.9722,  1.6144, 1.6144
]


29-element Vector{Float64}:
 2.8798
 2.9671
 2.7576
 2.8798
 0.5236
 0.2618
 2.8798
 0.5236
 2.7576
 2.8798
 0.5236
 0.2618
 2.618
 ⋮
 2.618
 2.0944
 1.9722
 1.6144
 1.6144
 2.6704
 1.5882
 2.618
 2.0944
 1.9722
 1.6144
 1.6144

In [ ]:
function optimize_trajectory_sparse(nx, nu, dt, N, x_eq, u_eq, equilib_foot_pos, kick_foot_pos, model, lower_joint_limits, upper_joint_limits, x_ref, u_ref)
    Q1 = diagm(1e0*ones(3))
    Q2 = diagm(1e0*ones(nx))
    R = diagm(1e-2*ones(nu))
    Qf = diagm(1e2*ones(3))
    x_ic = 1 * x_eq
    x_ic[1:3] .= 0
    x_g = 1 * x_eq
    x_g[1:3] .= 0

    # introduce the slack variables for the constraints
    components = (
        x = rand(nx, N),
        u = rand(nu, N),
        s_p_dynamics = rand(nx, N),
        s_n_dynamics = rand(nx, N),
        s_p_ic = rand(nx, N),
        s_n_ic = rand(nx, N),
        # s_p_g = rand(nx, N),
        # s_n_g = rand(nx, N),
        s_p_foot = rand(3, N),
        s_n_foot = rand(3, N)
    )

    z0 = vcat(
        [vcat(x_ref[i], u_ref[i], rand(nx), rand(nx), rand(nx), rand(nx), rand(3), rand(3)) for i in 1:N]...
    )
   
    #z0  = vcat([x_ref[i], u_ref[i], rand(nx), rand(nx), rand(nx), rand(nx), rand(3), rand(3) for i in 1:N]...)
    # push!(z0, vcat(x_ref[N], u_ref[N-1], rand(nx), rand(nx), rand(nx), rand(nx), rand(3), rand(3)))

    # foot_ref = vcat([
    #     (1 - t) * equilib_foot_pos + t * kick_foot_pos for t in range(0, stop=1, length=div(N, 2))
    # ],
    # [
    #     (1 - t) * kick_foot_pos + t * equilib_foot_pos for t in range(0, stop=1, length=div(N, 2))
    # ])

    foot_ref = [(1 - t) * equilib_foot_pos + t * kick_foot_pos for t in LinRange(0, 1, N)]


    #foot_pos = foot_equilib_pos .+ 0.1 * rand(3)
    traj = NamedTrajectory(components; timestep=dt, controls=:u)
    traj_indices = NamedTuple{traj.names}([[(k - 1)*traj.dim .+ getproperty(traj.components, symbol) for k in 1:traj.T] for symbol in traj.names])
    cost_objs = vcat(
        [quadratic_cost(traj_indices, k, Q1, Q2, R, x_g, foot_ref, model.mech, N÷2) for k = 1:N-1], 
        final_cost(traj_indices, N, Qf, foot_ref, mech, x_g)
    )
    # con_objs = Vector{NamedTuple}([foot_position_constraint(traj_indices, model.mech, kick_foot_pos, N÷2), ic_constraint(traj_indices, x_ic), goal_constraint(traj_indices, x_g, N), [dyn_constraint(traj_indices, k) for k = 1:N-1]...])
    con_objs = Vector{NamedTuple}([foot_position_constraint(traj_indices, model.mech, kick_foot_pos, N), ic_constraint(traj_indices, x_ic), [dyn_constraint(traj_indices, k) for k = 1:N-1]...])

    nc, conjac = setup_constraints(traj, con_objs)

    # u_scale is used to normalize u to be closer to 1 (allowing for faster optimization)
    param = (costs = cost_objs, constraints = con_objs, nconstraints=nc, nz=length(traj.datavec), model=model, dt=dt)
    @assert nc < length(traj.datavec)
    # Constrain bounds (equality and inequality)
    c_l, c_u = constraint_bounds(param)


    # Intial_guess
    #z0 = randn(param.nz) * 0.01
    #z0 = vcat([[x_eq; u_eq] for _ in 1:N]...) # warm start

    # primal bounds
    z_l, z_u = fill(-120.0, param.nz), fill(120.0, param.nz)


    # bound foot rotation to be zero
    for k = 1:N
        xi = traj_indices.x[k]
        ui = traj_indices.u[k]
        s_p_dynamics_i = traj_indices.s_p_dynamics[k]
        s_n_dynamics_i = traj_indices.s_n_dynamics[k]

        z_l[xi[1:3]] .= 0
        z_u[xi[1:3]] .= 0
        z_l[xi[4:32]] .= lower_joint_limits
        z_u[xi[4:32]] .= upper_joint_limits
        z_l[xi[36:64]] .= -15
        z_u[xi[36:64]] .= 15
        z_l[s_p_dynamics_i] .= 0
        z_l[s_n_dynamics_i] .= 0
        z_u[s_p_dynamics_i] .= Inf
        z_u[s_n_dynamics_i] .= Inf

        z_l[traj_indices.s_p_ic[k]] .= 0
        z_u[traj_indices.s_p_ic[k]] .= Inf
        z_l[traj_indices.s_n_ic[k]] .= 0
        z_u[traj_indices.s_n_ic[k]] .= Inf
    
        # z_l[traj_indices.s_p_g[k]] .= 0
        # z_u[traj_indices.s_p_g[k]] .= Inf
        # z_l[traj_indices.s_n_g[k]] .= 0
        # z_u[traj_indices.s_n_g[k]] .= Inf
    
        z_l[traj_indices.s_p_foot[k]] .= 0
        z_u[traj_indices.s_p_foot[k]] .= Inf
        z_l[traj_indices.s_n_foot[k]] .= 0
        z_u[traj_indices.s_n_foot[k]] .= Inf
    end

    z = lazy_nlp_qd.sparse_fmincon(cost_func,
                                cost_gradient!,
                                constraint_residual!,
                                constraint_jacobian!,
                                conjac,
                                z_l,
                                z_u, 
                                c_l,
                                c_u,
                                z0,
                                param,
                                tol = 1e-1, # for testing purposes
                                c_tol = 1e-1, # for testing purposes
                                max_iters = 100,
                                print_level = 5); # for testing purposes
    traj.datavec .= z
    return traj
end

optimize_trajectory_sparse (generic function with 1 method)

In [ ]:
#Pkg.add("FileIO")
# Save full trajectory data
model = G1Humanoid()
mech = model.mech
nx, nu, dt, N = size(x_eq, 1), size(u_eq, 1), 0.2, 10 #length(states)

equilib_foot_pos = [-0.3697416851162835, -0.1812921683831133, 0.28428794901541954]
foot_pos = goal_foot_pos
x_init = states
u_init = [0.001 * rand(nu) for k=1:length(x_init)]
traj = optimize_trajectory_sparse(nx, nu, dt, N, x_eq, u_eq, equilib_foot_pos, foot_pos, model, lower_limits, upper_limits, x_init, u_init)
#CairoMakie.plot(z)
# i = 1
# for foot_pos in kick_arc
#     if i == 1
#         z0 = vcat([[x_eq; u_eq] for _ in 1:N]...)
#     else
#         traj_loaded = load("trajectory_$(i-1).jld2")
#         z0 = traj_loaded["z"]
#     end
#     println("Optimizing for index $(i), foot position: $(foot_pos)")
#     z = optimize_trajectory_sparse(nx, nu, dt, N, x_eq, u_eq, foot_pos, model, z0)
#     save("trajectory_$(i).jld2", "z", z, "traj_indices", traj_indices)
#     i += 1
# end
# Later reload


(x = UnitRange{Int64}[1:64, 356:419, 711:774, 1066:1129, 1421:1484, 1776:1839, 2131:2194, 2486:2549, 2841:2904, 3196:3259], u = UnitRange{Int64}[65:93, 420:448, 775:803, 1130:1158, 1485:1513, 1840:1868, 2195:2223, 2550:2578, 2905:2933, 3260:3288], s_p_dynamics = UnitRange{Int64}[94:157, 449:512, 804:867, 1159:1222, 1514:1577, 1869:1932, 2224:2287, 2579:2642, 2934:2997, 3289:3352], s_n_dynamics = UnitRange{Int64}[158:221, 513:576, 868:931, 1223:1286, 1578:1641, 1933:1996, 2288:2351, 2643:2706, 2998:3061, 3353:3416], s_p_ic = UnitRange{Int64}[222:285, 577:640, 932:995, 1287:1350, 1642:1705, 1997:2060, 2352:2415, 2707:2770, 3062:3125, 3417:3480], s_n_ic = UnitRange{Int64}[286:349, 641:704, 996:1059, 1351:1414, 1706:1769, 2061:2124, 2416:2479, 2771:2834, 3126:3189, 3481:3544], s_p_foot = UnitRange{Int64}[350:352, 705:707, 1060:1062, 1415:1417, 1770:1772, 2125:2127, 2480:2482, 2835:2837, 3190:3192, 3545:3547], s_n_foot = UnitRange{Int64}[353:355, 708:710, 1063:1065, 1418:1420, 1773:1775, 21

In [54]:
loaded = load("trajectory.jld2")
traj = loaded["traj"]
traj_indices = loaded["traj_indices"]

(x = UnitRange{Int64}[1:64, 94:157, 187:250, 280:343, 373:436, 466:529, 559:622, 652:715, 745:808, 838:901  …  2605:2668, 2698:2761, 2791:2854, 2884:2947, 2977:3040, 3070:3133, 3163:3226, 3256:3319, 3349:3412, 3442:3505], u = UnitRange{Int64}[65:93, 158:186, 251:279, 344:372, 437:465, 530:558, 623:651, 716:744, 809:837, 902:930  …  2669:2697, 2762:2790, 2855:2883, 2948:2976, 3041:3069, 3134:3162, 3227:3255, 3320:3348, 3413:3441, 3506:3534])

In [55]:
mvis = initialize_visualizer(model) # visualizer
open(mvis)
dt = 0.05
for k = 1:length(states)
    set_configuration!(mvis, traj.datavec[traj_indices.x[k][1:32]])
    sleep(dt)
    render(mvis)
end

[ Info: Listening on: 127.0.0.1:8740, thread id: 1
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8740
